In [ ]:
####### CONFIGURATION

class CFG:

    # environment
    environment = 'local'  # work environment ('kaggle', 'colab', 'local')
    device      = 'CPU' # device ('CPU', 'GPU', 'TPU')
    num_workers = 4        # no. cores

    # general
    version  = 128   # notebook version (for saving outputs)
    debug    = False # debug mode runs 5 batches for 2 epochs 
    training = False # if False, only inference is run
    tracking = False # track results using neptune.ai
    seed     = 13353 # random state

    # data
    num_folds  = 5      # number of CV folds
    data_2019  = True   # append 2019 labeled data to training folds (1a)
    data_pl    = 0.2    # False or percentage of appended pseudo-labeled data (1b)
    data_ext   = False  # False or list with external dataset ids (2, 3, 4, 5)
    drop_dupl  = True   # drop duplicate images from training folds
    drop_outs  = False  # drop outliers from training folds
    oversample = False  # enable oversampling through WeightedRandomSampler()
    
    # label noise
    drop_noise = False  # False or percentage of removed noisy labels in training folds
    flip_noise  = False  # False or percentage of flipped noisy labels in training folds

    # image size and augmentations 
    image_size   = 512                # image size after random crop
    crop_scale   = (0.10, 1)          # min scale, max scale
    gr_shuffle     = (3, 3)             # number of tiles in shuffled grid
    ssr          = [0.05, 0.05, 360]  # shift, scale, rotation limits
    huesat       = [20, 20, 20]       # hue, saturation, value limits
    bricon       = [0.1, 0.1]         # brightness, contrast limits
    blur_limit   = 3                  # blur limit
    dist_limit   = 0.1                # distortion limit
    cutout       = [5, 0.1]           # number of squares, size of squares
    p_augment    = 0.5                # prob. of augmentations except for flips
    cutmix       = [0, 1]             # cutmix batch-level probability, alpha
    normalize    = False              # pixel normalization (False, 'dataset', 'imagenet')
    pairing      = False              # overlay of a random image from the same class
    pairing_prob = 0.6                # probability of an image to get another image added
    pixel_crop   = 150                # hight and width of the crop

    # model architecture
    backbone = 'tf_efficientnet_b4_ns'  # convolutional backbone
    weights  = 'imagenet'              # weights ('empty', 'imagenet', 'custom')
    save_all = False                   # save weights from each epoch

    # pretrained model
    pr_version     = 3   # notebook version (2, 3, 4, 5)
    pr_num_classes = 10  # no. classes (2: 4, 3: 10, 4: 2, 5: 17)

    # training
    batch_size  = 32    # no. images per batch
    num_epochs  = 10    # no. epochs per fold
    fine_tune    = 2     # fine-tuning dense layers (False or no. epochs)
    accum_iter  = 1     # no. batches for gradient accumalation
    use_amp     = True  # automated mixed precision mode

    # learning rate and optimizer
    eta     = 1e-4    # starting learning rate
    eta_min = 1e-6    # ending learning rate
    optim   = 'AdamP' # LR optimizer ('Adam', 'AdamW', 'AdamP')
    decay   = 0       # weight decay of optimizer (L2 regularization)

    # learning rate scheduler
    warmup          = 1                  # no. epochs for warmup
    schedule        = 'CosineAnnealing'  # LR scheduler after warmup
    update_on_batch = True               # update LR after every batch or epoch

    # loss function
    loss_fn     = 'OHEM'       # loss function ('CE', 'OHEM', 'SCE', 'CCE', 'Focal', 'FocalCosine', 'Taylor', 'BiTempered')
    smoothing   = 0.2          # label smoothing (works with all losses)
    ohem        = 0.8          # OHEM loss parameters: top-k percentage
    sce         = [0.1, 1.0]   # SCE loss parameters: alpha, beta
    cce         = 5            # CCE loss parameters: gamma
    focal       = [1, 2]       # Focal loss parameters: alpha, gamma
    focalcosine = [1, 2, 0.1]  # FocalCosine loss parameters: alpha, gamma, xent
    taylor      = 2            # Taylor loss parameters: n
    bitempered  = [0.3, 1.0]   # BiTempered loss parameters: t1, t2

    # epoch-based changes
    step_size  = False  # False or list with image_size multiplier for each epoch
    step_class = False  # False or list with num_classes for each epoch (2 or 5)
    step_p_aug = False  # False or list with p_augment multiplier for each epoch
    step_loss  = False  # False or list with loss functions for each epoch

    # inference
    num_tta = 1  # no. TTA flips (between 1 and 8)
    
    data_path = '../../data/'
    
    
CFG = dict(vars(CFG))
for key in ['__dict__', '__doc__', '__module__', '__weakref__']:
    del CFG[key]

In [ ]:
#### Packages
import numpy as np
import pandas as pd

import random
import math
import time
import sys
import os
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import RandomSampler, SequentialSampler, WeightedRandomSampler
from torch.utils.data.distributed import DistributedSampler
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, ReduceLROnPlateau
from torch.autograd import Function

os.system('pip install timm')
import timm
from timm.utils import *

from contextlib import suppress

os.system('pip install --upgrade -U albumentations')
import albumentations as A
from albumentations.pytorch import ToTensorV2

os.system("pip install git+'https://github.com/ildoonet/pytorch-gradual-warmup-lr.git'")
from warmup_scheduler import GradualWarmupScheduler  

os.system('pip install adamp')
from adamp import AdamP

from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import cv2

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold, StratifiedKFold
from scipy.special import softmax

from tqdm import tqdm

import gc

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

device = torch.device('cpu')

####### UTILITIES

##### RANDOM FOR LOOPS

def randomly(seq):
    shuffled = list(seq)
    random.shuffle(shuffled)
    return iter(shuffled)


##### DEVICE-AWARE PRINTING 

def smart_print(expression, CFG):
    if CFG['device'] != 'TPU':
        print(expression)
    else:
        xm.master_print(expression)


##### DEVICE-AWARE SAVING

def smart_save(weights, path, CFG):
    if CFG['device'] != 'TPU':
        torch.save(weights, path)    
    else:
        xm.save(weights, path) 

    
##### RANDOMNESS

def seed_everything(seed, CFG):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    smart_print('Setting random seed to {}...'.format(seed), CFG)
    
seed_everything(CFG['seed'], CFG)

In [ ]:
####### 2020 COMPETITION DATA

# import data
df = pd.read_csv(CFG['data_path'] + 'train_with_imagenet_label.csv')
df = df.loc[df['source'] == 2020].reset_index(drop = True)
    
# num classes
CFG['num_classes'] = df['label'].nunique()

In [ ]:
####### DATASET

class LeafData(Dataset):
    
    # initialization
    def __init__(self, 
                 data, 
                 directory, 
                 transform    = None, 
                 labeled      = False,
                 pairing      = False,
                 pairing_prob = 0.01,
                 pixel_crop   = 50,
                 seed         = 0):
        
        self.data         = data
        self.directory    = directory
        self.transform    = transform
        self.labeled      = labeled
        self.pairing      = pairing
        self.pairing_prob = pairing_prob
        self.seed         = seed
        self.counter      = 0
        self.pixel_crop   = pixel_crop
        if pairing:
            self.sampled_idx = self.data.sample(frac=self.pairing_prob, random_state=self.seed).index
        
    # length
    def __len__(self):
        return len(self.data)
    
    # get item  
    def __getitem__(self, idx):
        
        # import
        path  = os.path.join(self.directory, self.data.iloc[idx]['image_id'])
        image = cv2.imread(path)
        if image is None:
            raise FileNotFoundError(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
        # augmentations
        if self.transform is not None:
            image = self.transform(image = image)['image']
            
        if self.pairing:
            if idx in self.sampled_idx:
                
                try:
                    add_path     = os.path.join(self.directory, self.data.loc[self.data.label==self.data.iloc[idx]['label']].sample(1, random_state=self.seed)['image_id'].iloc[0])
                except:
                    print(self.data.sample(1, random_state=self.seed))
                    add_path     = os.path.join(self.directory, self.data.sample(1, random_state=self.seed)['image_id'].iloc[0])
                    self.counter += 1
                    
                add_image = cv2.imread(add_path)
                add_image = cv2.cvtColor(add_image, cv2.COLOR_BGR2RGB)
                if self.transform is not None:
                    add_image = self.transform(image = add_image)['image']
                    
                max_x = add_image.shape[2] - self.pixel_crop
                max_y = add_image.shape[1] - self.pixel_crop
                
                x = torch.randint(0, max_x, (1,))
                y = torch.randint(0, max_y, (1,))
                crop  = add_image[:, y: y + self.pixel_crop, x: x + self.pixel_crop]
                image[:, y: y + self.pixel_crop, x: x + self.pixel_crop] = crop
                
        
        # output
        if self.labeled:
            labels = torch.tensor(self.data.iloc[idx]['label']).long()
            return image, labels
        else:
            return image
        
        def get_counter(self):
            print(f'{self.counter} observations were cropped with the sample from another class')

In [ ]:
####### AUGMENTATIONS

def get_augs(CFG, image_size = None, p_augment = None):

    # update epoch-based parameters
    if image_size is None:
        image_size = CFG['image_size']
    if p_augment is None:
        p_augment = CFG['p_augment']

    # normalization
    if CFG['normalize']:
        if CFG['normalize'] == 'dataset':
            CFG['pixel_mean'] = (0.442, 0.511, 0.318)
            CFG['pixels_std'] = (0.233, 0.236, 0.225)
        elif CFG['normalize'] == 'imagenet':
            CFG['pixel_mean'] = (0.485, 0.456, 0.406)
            CFG['pixels_std'] = (0.229, 0.224, 0.225)
    else:
        CFG['pixel_mean'] = (0, 0, 0)
        CFG['pixels_std'] = (1, 1, 1)

    # train augmentations
    train_augs = A.Compose([A.RandomResizedCrop(height = image_size, 
                                                width  = image_size,
                                                scale  = CFG['crop_scale']),
                            A.RandomGridShuffle(p    = p_augment,
                                              grid = CFG['gr_shuffle']),
                            A.Transpose(p = 0.5),
                            A.HorizontalFlip(p = 0.5),
                            A.VerticalFlip(p = 0.5),
                            A.ShiftScaleRotate(p            = p_augment,
                                               shift_limit  = CFG['ssr'][0],
                                               scale_limit  = CFG['ssr'][1],
                                               rotate_limit = CFG['ssr'][2]),
                            A.HueSaturationValue(p               = p_augment,
                                                 hue_shift_limit = CFG['huesat'][0],
                                                 sat_shift_limit = CFG['huesat'][1],
                                                 val_shift_limit = CFG['huesat'][2]),
                            A.RandomBrightnessContrast(p                = p_augment,
                                                       brightness_limit = CFG['bricon'][0],
                                                       contrast_limit   = CFG['bricon'][1]),
                            A.OneOf([A.MotionBlur(blur_limit   = CFG['blur_limit']),
                                     A.MedianBlur(blur_limit   = CFG['blur_limit']),
                                     A.GaussianBlur(blur_limit = CFG['blur_limit'])], 
                                     p = p_augment),
                            A.OneOf([A.OpticalDistortion(distort_limit = CFG['dist_limit']),
                                     A.GridDistortion(distort_limit    = CFG['dist_limit'])], 
                                     p = p_augment),
                            A.Cutout(p          = p_augment, 
                                     num_holes  = CFG['cutout'][0], 
                                     max_h_size = np.int(CFG['cutout'][1] * image_size), 
                                     max_w_size = np.int(CFG['cutout'][1] * image_size)),
                            A.Normalize(mean = CFG['pixel_mean'],
                                        std  = CFG['pixels_std']),
                            ToTensorV2()
                           ])

    # test augmentations
    test_augs = A.Compose([A.SmallestMaxSize(max_size = image_size),
                           A.CenterCrop(height = image_size, 
                                        width  = image_size),
                           A.Normalize(mean = CFG['pixel_mean'],
                                       std  = CFG['pixels_std']),
                           ToTensorV2()
                           ])
    
    # output
    return train_augs, test_augs

In [ ]:
df_freq = df.imagenet_label.value_counts()

In [ ]:
_, test_augs = get_augs(CFG, image_size = CFG['image_size'], p_augment = CFG['p_augment'])


    
for i,label in enumerate(list(df_freq.index)):
    if os.path.exists(f'{CFG["data_path"]}imagenet_classes/{label}.png'):
        continue
    else:
        if df_freq[label]<5:
            fig, axs = plt.subplots(1, df_freq[label], facecolor='w', edgecolor='k', figsize = (100, 40))
        else:
            fig, axs = plt.subplots(1,5, facecolor='w', edgecolor='k', figsize = (100, 40))
        
        if df_freq[label]!=1:
            axs = axs.ravel()

        train_dataset =  LeafData(data    = df.loc[df.imagenet_label==label,:].head(5), 
                             directory    = CFG['data_path'] + 'train_images/',
                             transform    = test_augs,
                             labeled      = True, 
                             seed         = CFG['seed'])

        train_loader  = DataLoader(dataset    = train_dataset, 
                                  batch_size  = 5, 
                                  shuffle     = False, 
                                  num_workers = 0,
                                  pin_memory  = True)
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            if df_freq[label]==1:
                for img_id, image in enumerate(inputs):
                    axs.imshow(image.permute(1, 2, 0))
                    axs.set_title(label)
            else:    
                for img_id, image in enumerate(inputs):
                    axs[img_id].imshow(image.permute(1, 2, 0))
                    axs[img_id].set_title(label)

        plt.savefig(f'{CFG["data_path"]}imagenet_classes/{label}.png')